In [1]:
# from google.colab import files
# uploaded = files.upload()
# Automatically download dataset from Google Drive
!pip install pathway

Importing data set

In [2]:
!pip show pathway

Name: pathway
Version: 0.24.1
Summary: Pathway is a data processing framework which takes care of streaming data updates for you.
Home-page: https://pathway.com/
Author: 
Author-email: 
License: BUSL-1.1
Location: /usr/local/lib/python3.11/dist-packages
Requires: aiobotocore, aiohttp, aiohttp-cors, async-lru, beartype, boto3, click, deltalake, diskcache, fs, geopy, gitpython, google-api-python-client, google-cloud-bigquery, google-cloud-pubsub, h3, jmespath, jupyter-bokeh, networkx, numpy, opentelemetry-api, opentelemetry-exporter-otlp-proto-grpc, opentelemetry-sdk, pandas, panel, pyarrow, pydantic, python-sat, requests, rich, scikit-learn, shapely, typing-extensions
Required-by: 


In [3]:

!pip install -q gdown
file_id = "1D479FLjp9aO3Mg8g6Lpj9oRViWacurA6"
file_name = "dataset.csv"

# Download the file
!gdown https://drive.google.com/uc?id={file_id} -O {file_name}


Downloading...
From: https://drive.google.com/uc?id=1D479FLjp9aO3Mg8g6Lpj9oRViWacurA6
To: /content/dataset.csv
100% 92.2k/92.2k [00:00<00:00, 4.64MB/s]


In [4]:
import pandas as pd
import pathway as pw
from bokeh.plotting import figure, show, output_notebook
output_notebook()


# Loading the uploaded CSV file
df = pd.read_csv("dataset.csv")

# Display first few rows
df.head()


,Unnamed: 0,SystemCodeNumber,Capacity,Occupancy,LastUpdatedDate,LastUpdatedTime,IsSpecialDay,VehicleType,Latitude,Longitude,TrafficConditionNearby,QueueLength
0,0,BHMBCCMKT01,577,61,04-10-2016,07:59:42,0,car,28.5,77.15,low,2
1,1,BHMBCCMKT01,577,64,04-10-2016,08:25:42,0,car,28.5,77.15,average,2
2,2,BHMBCCMKT01,577,80,04-10-2016,08:59:42,0,car,28.5,77.15,low,2
3,3,BHMBCCMKT01,577,107,04-10-2016,09:32:46,0,car,28.5,77.15,low,3
4,4,BHMBCCMKT01,577,150,04-10-2016,09:59:48,0,car,28.5,77.15,low,3


Date and time combining

In [5]:
# Combine 'LastUpdatedDate' and 'LastUpdatedTime' into one datetime column
df["Datetime"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"],
                                 format="%d-%m-%Y %H:%M:%S")

# Drop the original columns since we now have one datetime
df = df.drop(columns=["LastUpdatedDate", "LastUpdatedTime"])

# Display updated dataframe
df.head()


,Unnamed: 0,SystemCodeNumber,Capacity,Occupancy,IsSpecialDay,VehicleType,Latitude,Longitude,TrafficConditionNearby,QueueLength,Datetime
0,0,BHMBCCMKT01,577,61,0,car,28.5,77.15,low,2,2016-10-04 07:59:42
1,1,BHMBCCMKT01,577,64,0,car,28.5,77.15,average,2,2016-10-04 08:25:42
2,2,BHMBCCMKT01,577,80,0,car,28.5,77.15,low,2,2016-10-04 08:59:42
3,3,BHMBCCMKT01,577,107,0,car,28.5,77.15,low,3,2016-10-04 09:32:46
4,4,BHMBCCMKT01,577,150,0,car,28.5,77.15,low,3,2016-10-04 09:59:48


Checking unique parking lots

In [6]:
# How many parking lots are there?
df["SystemCodeNumber"].nunique()


1

Converting Vehicle and Traffic into categorical data

In [7]:
# Convert categorical text into numbers
df["VehicleType"] = df["VehicleType"].map({"bike": 0, "car": 1, "truck": 2})
df["TrafficConditionNearby"] = df["TrafficConditionNearby"].map({"low": 0, "medium": 1, "high": 2})

# Confirm changes
df[["VehicleType", "TrafficConditionNearby"]].head()


,VehicleType,TrafficConditionNearby
0,1.0,0.0
1,1.0,NaN
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0


Inspecting dataset

In [8]:
# Check for missing/null values
df.isnull().sum()


,0
Unnamed: 0,0
SystemCodeNumber,0
Capacity,0
Occupancy,0
IsSpecialDay,0
VehicleType,140
Latitude,0
Longitude,0
TrafficConditionNearby,446
QueueLength,0


Outlier & invalid data check

In [9]:
# Check if occupancy exceeds capacity
invalid_occupancy = df[df["Occupancy"] > df["Capacity"]]
invalid_occupancy


,Unnamed: 0,SystemCodeNumber,Capacity,Occupancy,IsSpecialDay,VehicleType,Latitude,Longitude,TrafficConditionNearby,QueueLength,Datetime


In [10]:
#invalid queue length check

invalid_queue = df[df["QueueLength"] < 0]
invalid_queue

,Unnamed: 0,SystemCodeNumber,Capacity,Occupancy,IsSpecialDay,VehicleType,Latitude,Longitude,TrafficConditionNearby,QueueLength,Datetime


In [11]:
#invalid vehicle check
invalid_vehicle_type = df[~df["VehicleType"].isin([0, 1, 2])]
invalid_vehicle_type

,Unnamed: 0,SystemCodeNumber,Capacity,Occupancy,IsSpecialDay,VehicleType,Latitude,Longitude,TrafficConditionNearby,QueueLength,Datetime
15,15,BHMBCCMKT01,577,165,0,NaN,28.5,77.15,NaN,3,2016-10-04 15:30:14
18,18,BHMBCCMKT01,577,54,0,NaN,28.5,77.15,0.0,1,2016-10-05 07:57:17
24,24,BHMBCCMKT01,577,148,0,NaN,28.5,77.15,NaN,6,2016-10-05 11:04:19
34,34,BHMBCCMKT01,577,121,0,NaN,28.5,77.15,0.0,3,2016-10-05 16:04:18
52,52,BHMBCCMKT01,577,135,0,NaN,28.5,77.15,0.0,2,2016-10-06 16:04:23
...,...,...,...,...,...,...,...,...,...,...,...
1279,1279,BHMBCCMKT01,577,8,1,NaN,28.5,77.15,0.0,1,2016-12-18 09:30:21
1281,1281,BHMBCCMKT01,577,44,1,NaN,28.5,77.15,2.0,7,2016-12-18 10:30:23
1290,1290,BHMBCCMKT01,577,303,1,NaN,28.5,77.15,NaN,8,2016-12-18 14:56:25
1296,1296,BHMBCCMKT01,577,24,0,NaN,28.5,77.15,0.0,2,2016-12-19 09:03:33


Dropping invalid rows

In [12]:
#invalid occupancy
df = df[df["Occupancy"] <= df["Capacity"]]
#invalid vehicle
df = df[df["VehicleType"].isin([0, 1, 2])]
#checking
print("Remaining rows after cleaning:", df.shape[0])


Remaining rows after cleaning: 1172


Pricing function

Model 1

Real time simulation

In [13]:
# import pandas as pd
# import pathway as pw

# # Confirm correct headers
# df = pd.read_csv("dataset.csv")
# print("CSV Columns:", df.columns.tolist())
# # Rename if needed
# if "Unnamed: 0" in df.columns:
#     df.rename(columns={"Unnamed: 0": "ID"}, inplace=True)
#     df.to_csv("dataset.csv", index=False)
# assert "ID" in df.columns, "ID column missing!"
# assert "Datetime" not in df.columns, "Remove Datetime from CSV!"

# # Optional: Ensure Unnamed: 0 is renamed (in case old file is loaded)
# df.rename(columns={"Unnamed: 0": "ID"}, inplace=True)
# df.to_csv("dataset.csv", index=False)

# # Define schema (NO Datetime here!)
# class ParkingSchema(pw.Schema):
#     ID: int
#     SystemCodeNumber: str
#     Capacity: int
#     Occupancy: int
#     LastUpdatedDate: str
#     LastUpdatedTime: str
#     IsSpecialDay: int
#     VehicleType: str
#     Latitude: float
#     Longitude: float
#     TrafficConditionNearby: str
#     QueueLength: int

# # UDFs
# @pw.udf
# def combine_datetime(date: str, time: str) -> str:
#     return f"{date} {time}"

# @pw.udf
# def compute_price(occupancy: int, capacity: int) -> float:
#     base_price = 10
#     alpha = 5
#     return round(base_price + alpha * (occupancy / capacity), 2) if capacity else base_price

# # Read CSV (no Datetime column here)
# input_table = pw.io.csv.read(
#     "dataset.csv",
#     schema=ParkingSchema,
#     mode="static"
# )

# # Select and compute new fields
# output_table = input_table.select(
#     input_table.ID,
#     input_table.SystemCodeNumber,
#     input_table.Capacity,
#     input_table.Occupancy,
#     input_table.VehicleType,
#     input_table.TrafficConditionNearby,
#     Datetime=combine_datetime(input_table.LastUpdatedDate, input_table.LastUpdatedTime),
#     price=compute_price(input_table.Occupancy, input_table.Capacity)
# )

# # Write output
# pw.io.jsonlines.write(output_table, filename="output.jsonl")
# pw.run()

Output()

/usr/local/lib/python3.11/dist-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Iterable[pathway.internals.expression.ColumnReference] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


CSV Columns: ['Unnamed: 0', 'SystemCodeNumber', 'Capacity', 'Occupancy', 'LastUpdatedDate', 'LastUpdatedTime', 'IsSpecialDay', 'VehicleType', 'Latitude', 'Longitude', 'TrafficConditionNearby', 'QueueLength']


In [14]:
# # with open("output.txt", "r") as f:
# #     for _ in range(5):
# #         print(f.readline().strip())
# # Read the output file content (to preview or confirm)
# with open("output.jsonl", "r") as file:
#     lines = file.readlines()
#     for line in lines[:5]:  # print first 5 lines for confirmation
#         print(line.strip())

{"ID":1123,"SystemCodeNumber":"BHMBCCMKT01","Capacity":577,"Occupancy":190,"VehicleType":"bike","TrafficConditionNearby":"average","Datetime":"09-12-2016 11:29:33","price":11.65,"diff":1,"time":1752084710586}
{"ID":348,"SystemCodeNumber":"BHMBCCMKT01","Capacity":577,"Occupancy":202,"VehicleType":"bike","TrafficConditionNearby":"average","Datetime":"25-10-2016 10:56:23","price":11.75,"diff":1,"time":1752084710586}
{"ID":836,"SystemCodeNumber":"BHMBCCMKT01","Capacity":577,"Occupancy":122,"VehicleType":"car","TrafficConditionNearby":"average","Datetime":"21-11-2016 11:57:30","price":11.06,"diff":1,"time":1752084710586}
{"ID":1063,"SystemCodeNumber":"BHMBCCMKT01","Capacity":577,"Occupancy":11,"VehicleType":"truck","TrafficConditionNearby":"low","Datetime":"06-12-2016 08:29:11","price":10.1,"diff":1,"time":1752084710586}
{"ID":964,"SystemCodeNumber":"BHMBCCMKT01","Capacity":577,"Occupancy":224,"VehicleType":"cycle","TrafficConditionNearby":"high","Datetime":"28-11-2016 13:02:26","price":11.

In [15]:
# import pandas as pd
# from bokeh.plotting import figure, show, output_file
# from bokeh.models import HoverTool

# # ✅ Load output data
# df = pd.read_json("output.jsonl", lines=True)

# # ✅ Output HTML file
# output_file("price_vs_occupancy.html")

# # ✅ Create figure
# p = figure(
#     title="Dynamic Parking Price vs Occupancy",
#     x_axis_label="Occupancy",
#     y_axis_label="Price",
#     width=800,
#     height=500,
#     tools="pan,wheel_zoom,box_zoom,reset,hover,save"
# )

# # ✅ Hover tool
# hover = p.select(dict(type=HoverTool))
# hover.tooltips = [
#     ("SystemCodeNumber", "@SystemCodeNumber"),
#     ("VehicleType", "@VehicleType"),
#     ("Traffic", "@TrafficConditionNearby"),
#     ("Datetime", "@Datetime"),
#     ("Occupancy", "@Occupancy"),
#     ("Price", "@price")
# ]

# # ✅ Use scatter instead of circle (Bokeh 3.4+)
# p.scatter(
#     x=df["Occupancy"],
#     y=df["price"],
#     size=8,
#     color="navy",
#     alpha=0.6,
#     marker="circle"
# )

# # ✅ Show the plot
# show(p)

Model 2

In [16]:
import pathway as pw
import math

# Mapping dictionaries
TRAFFIC_LEVELS = {"low": 1, "average": 2, "high": 3}
VEHICLE_WEIGHTS = {"bike": 0.5, "car": 1.0, "truck": 1.5, "cycle": 0.3}

# Schema matching the corrected CSV file (with "ID" column now)
class ParkingSchema(pw.Schema):
    ID: int
    SystemCodeNumber: str
    Capacity: int
    Occupancy: int
    LastUpdatedDate: str
    LastUpdatedTime: str
    IsSpecialDay: int
    VehicleType: str
    Latitude: float
    Longitude: float
    TrafficConditionNearby: str
    QueueLength: int

# Read the CSV
input_table = pw.io.csv.read(
    "dataset.csv",
    schema=ParkingSchema,
    mode="static"
    # mode="streaming",  # enables streaming mode
    # autocommit_duration_ms=1000       # simulates data coming every 1 second (adjustable)
)

# Demand-based pricing UDF
@pw.udf
def compute_demand_based_price(occupancy: int, capacity: int, queue_length: int,
                                traffic: str, is_special: int, vehicle_type: str) -> float:
    base_price = 10
    λ = 0.5

    # Weights
    α = 2.0
    β = 0.5
    γ = 1.0
    δ = 1.0
    ε = 1.0

    # Safe get values
    traffic_weight = TRAFFIC_LEVELS.get(traffic.lower(), 2)
    vehicle_weight = VEHICLE_WEIGHTS.get(vehicle_type.lower(), 1.0)

    # Demand calculation
    occ_rate = occupancy / capacity if capacity != 0 else 0
    demand = (
        α * occ_rate +
        β * queue_length -
        γ * traffic_weight +
        δ * is_special +
        ε * vehicle_weight
    )

    # Normalize and clamp
    normalized_demand = 1 / (1 + math.exp(-demand))
    raw_price = base_price * (1 + λ * normalized_demand)
    final_price = max(min(raw_price, 2.0 * base_price), 0.5 * base_price)

    return round(final_price, 2)

# Apply Model 2 to input
output_table = input_table.select(
    input_table.ID,
    input_table.SystemCodeNumber,
    input_table.Capacity,
    input_table.Occupancy,
    input_table.QueueLength,
    input_table.TrafficConditionNearby,
    input_table.IsSpecialDay,
    input_table.VehicleType,
    price_model2=compute_demand_based_price(
        input_table.Occupancy,
        input_table.Capacity,
        input_table.QueueLength,
        input_table.TrafficConditionNearby,
        input_table.IsSpecialDay,
        input_table.VehicleType
    )
)

# Save result
pw.io.jsonlines.write(output_table, filename="output_model2.jsonl")

# Run the Pathway pipeline
pw.run()

# If in Google Colab, download result
# from google.colab import files
# files.download("output_model2.jsonl")

Output()

In [17]:
with open("output_model2.jsonl", "r") as file:
    lines = file.readlines()
    for line in lines[:5]:  # print first 5 lines for confirmation
        print(line.strip())

{"ID":1123,"SystemCodeNumber":"BHMBCCMKT01","Capacity":577,"Occupancy":190,"QueueLength":6,"TrafficConditionNearby":"average","IsSpecialDay":0,"VehicleType":"bike","price_model2":14.48,"diff":1,"time":1752084711032}
{"ID":348,"SystemCodeNumber":"BHMBCCMKT01","Capacity":577,"Occupancy":202,"QueueLength":5,"TrafficConditionNearby":"average","IsSpecialDay":0,"VehicleType":"bike","price_model2":14.23,"diff":1,"time":1752084711032}
{"ID":836,"SystemCodeNumber":"BHMBCCMKT01","Capacity":577,"Occupancy":122,"QueueLength":6,"TrafficConditionNearby":"average","IsSpecialDay":0,"VehicleType":"car","price_model2":14.59,"diff":1,"time":1752084711032}
{"ID":1063,"SystemCodeNumber":"BHMBCCMKT01","Capacity":577,"Occupancy":11,"QueueLength":0,"TrafficConditionNearby":"low","IsSpecialDay":0,"VehicleType":"truck","price_model2":13.16,"diff":1,"time":1752084711032}
{"ID":964,"SystemCodeNumber":"BHMBCCMKT01","Capacity":577,"Occupancy":224,"QueueLength":9,"TrafficConditionNearby":"high","IsSpecialDay":0,"Veh

In [18]:
import pandas as pd
from bokeh.plotting import figure, show, output_file
from bokeh.models import HoverTool

# ✅ Load Model 2 output
df = pd.read_json("output_model2.jsonl", lines=True)

# ✅ Output HTML file
output_file("model2_price_vs_occupancy.html")

# ✅ Create interactive plot
p = figure(
    title="Model 2: Demand-Based Price vs Occupancy",
    x_axis_label="Occupancy",
    y_axis_label="Price (Model 2)",
    width=800,
    height=500,
    tools="pan,wheel_zoom,box_zoom,reset,hover,save"
)

# ✅ Configure hover tool
hover = p.select(dict(type=HoverTool))
hover.tooltips = [
    ("ID", "@ID"),
    ("SystemCodeNumber", "@SystemCodeNumber"),
    ("Occupancy", "@Occupancy"),
    ("QueueLength", "@QueueLength"),
    ("VehicleType", "@VehicleType"),
    ("Traffic", "@TrafficConditionNearby"),
    ("Price", "@price_model2")
]

# ✅ Plot using scatter
p.scatter(
    x=df["Occupancy"],
    y=df["price_model2"],
    size=8,
    color="green",
    alpha=0.6,
    marker="circle"
)

# ✅ Show the plot
show(p)